In [ ]:
## 杀旧代码运行的残留结果
# ===== 一键“clc + clear all + close all” =====
import os
import sys
try:
    from IPython import get_ipython
    get_ipython().magic('clear')        # 类似 clc：清空终端/控制台
    get_ipython().magic('reset -f')     # 类似 clear all：清掉用户变量
except:
    # 非 IPython 环境，退而求其次
    os.system('cls' if os.name == 'nt' else 'clear')  # 清屏
    # 清掉当前全局变量（保留内置 __builtins__ 等）
    for name in dir():
        if not name.startswith('_') and name not in {'sys', 'os'}:
            del globals()[name]

# 如果有 GUI 窗口，把 matplotlib 的图全关掉
try:
    import matplotlib.pyplot as plt
    plt.close('all')
except ImportError:
    pass
# ===========================================

### 前置环境配置，库、包导入，自定义函数导入

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 设置中文字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False


##### 大图适配屏幕（2560×1440） + 放大镜

In [ ]:

# ==========================================================
# 通用：大图适配屏幕（2560×1440） + 放大镜
# ==========================================================
def show_with_zoom(title, img,
                   screen_wh=(2560, 1440),
                   roi_half=60,          # 放大镜取样半径
                   zoom_step=0.2):       # 滚轮步长
    """
    将 img 等比例缩放并在 screen_wh 窗口中居中显示，
    同时支持鼠标悬浮局部放大，滚轮调节放大倍率。
    按任意键退出。
    """
    scr_w, scr_h = screen_wh
    h0, w0 = img.shape[:2]

    # 1. 等比例缩放
    scale = min(scr_w / w0, scr_h / h0, 1.0)
    new_w, new_h = int(w0 * scale), int(h0 * scale)
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # 2. 黑边
    top = (scr_h - new_h) // 2
    bottom = scr_h - new_h - top
    left = (scr_w - new_w) // 2
    right = scr_w - new_w - left
    canvas = cv2.copyMakeBorder(resized, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=(0, 0, 0))
    clone = canvas.copy()

    # 3. 放大镜参数
    zoom = 3.0

    # 4. 鼠标回调
    def mouse_cb(event, x, y, flags, param):
        nonlocal zoom
        if event == cv2.EVENT_MOUSEMOVE:
            canvas[:] = clone[:]   # 复原
            x1 = max(x - roi_half, 0)
            y1 = max(y - roi_half, 0)
            x2 = min(x + roi_half, canvas.shape[1])
            y2 = min(y + roi_half, canvas.shape[0])
            if x2 - x1 == 0 or y2 - y1 == 0:
                return
            roi = clone[y1:y2, x1:x2]
            zoomed = cv2.resize(roi, (0, 0), fx=zoom, fy=zoom,
                                interpolation=cv2.INTER_LINEAR)
            zh, zw = zoomed.shape[:2]
            canvas[0:zh, canvas.shape[1]-zw:canvas.shape[1]] = zoomed
            cv2.imshow(title, canvas)

        elif event == cv2.EVENT_MOUSEWHEEL:
            zoom = max(1.0, zoom + zoom_step * (1 if flags > 0 else -1))

    # 5. 窗口
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(title, scr_w, scr_h)
    cv2.setMouseCallback(title, mouse_cb)
    cv2.imshow(title, canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


##### ROI框选函数

In [ ]:
def pick_roi_scaled_center(title, src_img, screen_wh=(2560, 1440)):
    scr_w, scr_h = screen_wh
    h0, w0 = src_img.shape[:2]

    # 1. 等比例缩放因子
    scale = min(scr_w / w0, scr_h / h0, 1.0)
    new_w, new_h = int(w0 * scale), int(h0 * scale)

    # 2. 缩放图
    small = cv2.resize(src_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # 3. 加黑边，使最终尺寸 = 屏幕分辨率
    top = (scr_h - new_h) // 2
    bottom = scr_h - new_h - top
    left = (scr_w - new_w) // 2
    right = scr_w - new_w - left
    canvas = cv2.copyMakeBorder(small, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 4. 创建窗口并强制 2560×1440
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(title, scr_w, scr_h)

    # 5. 状态变量
    roi = None
    drawing = False
    ix, iy = -1, -1

    def on_mouse(event, x, y, flags, param):
        nonlocal roi, ix, iy, drawing
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            ix, iy = x, y
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            tmp = canvas.copy()
            cv2.rectangle(tmp, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow(title, tmp)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            x1, y1 = min(ix, x), min(iy, y)
            x2, y2 = max(ix, x), max(iy, y)
            roi = (x1, y1, x2 - x1, y2 - y1)   # (x, y, w, h) on canvas

    cv2.imshow(title, canvas)
    cv2.setMouseCallback(title, on_mouse)

    # 6. 等待用户框选
    while True:
        k = cv2.waitKey(1) & 0xFF
        if roi is not None or k == 27:   # ROI 完成或按 ESC 退出
            break
    cv2.destroyWindow(title)

    # 7. 如果用户没框选，默认全图
    if roi is None:
        roi = (left, top, new_w, new_h)

    # 8. ROI 坐标映射回原图
    rx, ry, rw, rh = roi
    rx -= left
    ry -= top
    rx = int(rx / scale)
    ry = int(ry / scale)
    rw = int(rw / scale)
    rh = int(rh / scale)

    # 越界保护
    rw = min(rw, w0 - rx)
    rh = min(rh, h0 - ry)
    return rx, ry, rw, rh

##### 从单应矩阵  H  快速算旋转角（平面内）

In [ ]:
## 从单应矩阵  H  快速算旋转角（平面内）
def compute_rotation_angle(H):
    """
    假设模板与场景都是平面，且相机垂直拍摄，
    只关心绕 Z 轴的旋转角（平面内旋转）。
    """
    # H 左上 2×2 子矩阵做极分解，得到旋转部分
    a, b = H[0, 0], H[0, 1]
    c, d = H[1, 0], H[1, 1]
    theta = np.arctan2(c, a)            # 弧度
    return np.degrees(theta)            # 转角度

#### 导入图片

In [ ]:
img1 = cv2.imread('20250811162741.jpg')
# img1 = cv2.imread('box.png')
img2 = cv2.imread('20250811162828.jpg')


##### 对原始图像进行ROI框选

In [ ]:
# ==========================================================
# 1. 模板图 img1
# ==========================================================
x1, y1, w1, h1 = pick_roi_scaled_center("Select ROI on template (img1)", img1)
img1 = img1[y1:y1+h1, x1:x1+w1]

# # ==========================================================
# # 2. 场景图 img2
# # ==========================================================
# x2, y2, w2, h2 = pick_roi_scaled_center("Select ROI on scene (img2)", img2)
# img2 = img2[y2:y2+h2, x2:x2+w2]

# 下面继续做灰度化、SIFT、匹配即可

##### 图像预处理

In [ ]:
# 灰度化
g1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
g2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#### SIFT提取特征点（描述子）

In [ ]:
# 创建 SIFT 并设置阈值
sift = cv2.xfeatures2d.SIFT_create(
    nfeatures=1200,          # 先放宽上限
    contrastThreshold=0.08,  # 默认 0.04，越大越严格
    edgeThreshold=30         # 默认 10，越大越抑制边缘点
)

# 检测关键点并计算描述子
kp1, des1 = sift.detectAndCompute(g1, None)
kp2, des2 = sift.detectAndCompute(g2, None)

# 按响应值排序并再次裁剪
max_kp = 400  # 最终保留数量，按需改
kp1 = sorted(kp1, key=lambda k: k.response, reverse=True)[:max_kp]
kp2 = sorted(kp2, key=lambda k: k.response, reverse=True)[:max_kp]

# 重新计算描述子（只保留筛选后的关键点）
kp1, des1 = sift.compute(g1, kp1)
kp2, des2 = sift.compute(g2, kp2)

print('img1 保留关键点:', len(kp1))
print('img2 保留关键点:', len(kp2))


# ---------- 1. 绘制关键点 ----------
cv2.drawKeypoints(g1, kp1, img1)

# ---------- 7. 显示 ----------
show_with_zoom('Keypoints', img1)  

#### 基于K临近的BF暴力匹配

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1)

# match = bf.match(des1, des2)##简单最佳匹配，不是很推荐
# # 1. 绘制匹配结果
# img3_raw = cv2.drawMatches(img1, kp1, img2, kp2, match, None)

bf = cv2.BFMatcher(cv2.NORM_L1)
# K近邻匹配
matches = bf.knnMatch(des1, des2, k=2)

# Lowe's ratio test
RATIO = 0.6  #Lowe 阈值，越低越严苛。Ratio的值见前面的代码。
good_matches = []
for m, n in matches:
    if m.distance < RATIO * n.distance:
        good_matches.append(m)          

#重新包装成二维 list，满足 drawMatchesKnn 要求
good_matches_2d = [[m] for m in good_matches]

# 1. 绘制匹配结果
img3_raw = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_2d, None,
                              flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
''' 
cv2.drawMatchesKnn  要求第二个参数必须是 “二维 list”
（即  [[m1], [m2], …] ），而  good_matches  直接放成一维 list  [m1, m2, …], 会报错SystemError
'''
show_with_zoom('BF Matches', img3_raw)

#### 图像配准

In [ ]:
RATIO   = RATIO        # Lowe 阈值，越低越严苛。Ratio的值见前面的代码。
RANSAC_THRESH = 3.0    # 像素误差，越低越严苛
MIN_INLIERS   = 15     # 至少这么多内点才算成功
MIN_MATCH     = max(MIN_INLIERS, 4)#防止用户把  MIN_INLIERS  设得太小导致  findHomography  崩溃。
''' 
max 函数取两者中的较大值。
因为  findHomography  最少需要 4 对点才能算出单应矩阵，所以即使你把  MIN_INLIERS  设得比 4 小，
也至少保证有 4 个匹配再进入后续步骤。
如果你把  MIN_INLIERS  设成 15、20 之类大于 4 的数， MIN_MATCH  就自动等于  MIN_INLIERS ，代码仍成立。
'''

# good_matches = []
# for m, n in matches:
#     if m.distance < RATIO * n.distance:
#         good_matches.append(m)          # 直接存 DMatch，省得再 1d

if len(good_matches) >= MIN_MATCH:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_pts, dst_pts,
                                 cv2.RANSAC,
                                 RANSAC_THRESH)

    inliers = mask.ravel().sum()
    if inliers >= MIN_INLIERS:
        symbol_of_OKNG_for_Class2 = 1#匹配成功的自变量=1，用于后面的代码自行判断是否执行
        print(f'匹配成功！内点 = {inliers}')
        #在  findHomography  之后 再生成“真正用于打分的那一份内点列表”。
        good_inliers = [good_matches[i] for i, msk in enumerate(mask) if msk]
        ''' 
        
        '''
        # 画框、显示……
            # 5. 模板四角 → 场景坐标
        hT, wT = img1.shape[:2]
        pts = np.float32([[0, 0], [wT, 0], [wT, hT], [0, hT]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, H)

    # 6. 在场景图上画绿色四边形
        S_vis = img2.copy()
        cv2.polylines(S_vis, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)

    # 7. 把模板贴到场景里看看效果
        warped = cv2.warpPerspective(img1, H, (img2.shape[1], img2.shape[0]))
        blend = cv2.addWeighted(S_vis, 0.6, warped, 0.4, 0)
        show_with_zoom('Result', blend)   # 把任何 ndarray 传进来即可
    else:
        symbol_of_OKNG_for_Class2 = 0#匹配成功的自变量=1，用于后面的代码自行判断是否执行
        print('内点太少，匹配失败')
else:
    symbol_of_OKNG_for_Class2 = 0#匹配成功的自变量=1，用于后面的代码自行判断是否执行
    print('匹配点太少，配准失败！')

In [ ]:
print(f'原始 good 匹配数: {len(good_matches)}')
print(f'RANSAC 内点数   : {inliers}')
print(f'内点平均距离    : {np.mean([good_matches[i].distance for i, msk in enumerate(mask) if msk]):.2f}')


#### 判断是否找到模板对应的物体

In [ ]:
# ---------- 只判断是否找到物体 ----------
INLIER_RATIO = 0.75      # ≥75 % 的 good 匹配是内点就认为找到了

ratio = inliers / len(good_matches)
found = ratio >= INLIER_RATIO

if found:
    print('✅ 物体已找到','匹配分数：',ratio)
    # 下面直接算旋转角
    # angle = compute_rotation_angle(H)   # 
    # print(f'旋转角 ≈ {angle:.1f}°')#误差 ±2° 以内，足够做后续逻辑。
    # 把分数写在结果图上
    ratio_score=round(ratio*100)
    score_text = f'Score: {ratio_score}'
    cv2.putText(blend, score_text,
            (20, 60),                     # 左上角位置
            cv2.FONT_HERSHEY_SIMPLEX,
            1.8, (0, 255, 255), 3, cv2.LINE_AA)
    show_with_zoom('Result', blend)
else:
    print('❌ 未找到物体')

##### 如果找到对应的物体，判断旋转角度

In [ ]:
## 从单应矩阵  H  快速算旋转角（平面内）
def compute_rotation_angle(H):
    """
    假设模板与场景都是平面，且相机垂直拍摄，
    只关心绕 Z 轴的旋转角（平面内旋转）。
    """
    # H 左上 2×2 子矩阵做极分解，得到旋转部分
    a, b = H[0, 0], H[0, 1]
    c, d = H[1, 0], H[1, 1]
    theta = np.arctan2(c, a)            # 弧度
    return np.degrees(theta)            # 转角度

angle = compute_rotation_angle(H)   # 
print(f'旋转角 ≈ {angle:.1f}°')#误差 ±2° 以内，足够做后续逻辑。